Everyone is busy downloading and there was no kernel that kind of does the initial exploration to see how the images look like and how would a simple work on this dataset. Hence I went forward and created one. This kernel does not save any file locally and just uses them on the fly during training using a keras generator. Time ranges around ~240secs for an epoch with total 1280 images on a ResNet50 (with very minimal number of classes). There may be a few mistakes, you never know.

Happy Kaggling!

In [ ]:
import requests
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import threading
import urllib
import cv2
import time

import keras
from keras.applications import ResNet50
from keras import backend as K
from keras import regularizers
from keras.engine.topology import Input
from keras.layers import Activation, Add, BatchNormalization, Concatenate, Conv2D, Dense, Flatten, GlobalMaxPooling2D, \
    Lambda, MaxPooling2D, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras.utils import Sequence
from sklearn.model_selection import train_test_split

from collections import Counter

import os
import tarfile

In [ ]:
def check_size(url):
    r = requests.get(url, stream=True)
    return int(r.headers['Content-Length'])

def download_file(url, filename, bar=True):
    """
    Helper method handling downloading large files from `url` to `filename`. Returns a pointer to `filename`.
    """
    try:
        chunkSize = 1024
        r = requests.get(url, stream=True)
        with open(filename, 'wb') as f:
            if bar:
                pbar = tqdm( unit="B", total=int( r.headers['Content-Length'] ) )
            for chunk in r.iter_content(chunk_size=chunkSize): 
                if chunk: # filter out keep-alive new chunks
                    if bar: 
                        pbar.update (len(chunk))
                    f.write(chunk)
        return filename
    except Exception as e:
        print(e)
        return
    
def download_image_cv2_urllib(url):
    """
    Modifying the url to download the 360p or 720p version actually slows it down. 
    """
    try:
        resp = urllib.request.urlopen(url)
        foo = np.asarray(bytearray(resp.read()), dtype="uint8")
        foo = cv2.imdecode(foo, cv2.IMREAD_COLOR)
        foo = cv2.resize(foo,(128, 128), interpolation=cv2.INTER_AREA)
        return foo
    except:
        return np.array([])
    

# DATA
## Let's download the train set first.
## and set some usefull values

In [ ]:
## Usefull values
DATAPATH = '/mnt/DATA/Python_Projects/google_landmarks_2019/data/'

URLTRAINFILE = "https://s3.amazonaws.com/google-landmark/metadata/train.csv"
TRAINFILE = 'train.csv'

URLATTRIBFILE = "https://s3.amazonaws.com/google-landmark/metadata/train_attribution.csv"
ATTRIBFILE = "train_attribution.csv"


In [ ]:
# Download test file
print(os.path.isfile(DATAPATH + TRAINFILE))

if os.path.isfile(DATAPATH + TRAINFILE) is False:
    download_file(URLTRAINFILE, DATAPATH + TRAINFILE )

In [ ]:
# download attribute file
print(os.path.isfile(DATAPATH + TRAINFILE))

if os.path.isfile(DATAPATH + ATTRIBFILE) is False:
    download_file(URLATTRIBFILE, DATAPATH + ATTRIBFILE )

In [ ]:
# download image files, for a downloaded file, extracts it and deletes the tar one
for i in range(100,500):
    extension = "00" + str(i+1)
    extension = extension[-3:]
    url = "https://s3.amazonaws.com/google-landmark/train/images_" + extension + ".tar"
    filename = "images_" + extension + ".tar"
    if os.path.isdir(DATAPATH + filename[:-3]) is False:
        if os.path.isfile(DATAPATH + filename) is False:
            download_file(url, DATAPATH + filename , False )
            print("File {} downloaded".format(filename))
        tar = tarfile.open(DATAPATH + filename)
        tar.extractall(DATAPATH)
        print("File {} extracted".format(filename))
        os.remove(DATAPATH + filename)

        


# It's time to load and beguin to work with data
## First loading DataFrames

In [ ]:
train = pd.read_csv(DATAPATH + TRAINFILE)
print(train.head())
print(train.shape)
print("Number of classes {}".format(len(train.landmark_id.unique())))

In [ ]:
attrib = pd.read_csv(DATAPATH + ATTRIBFILE)
print(attrib.head())
print(attrib.shape)
print("Number of classes {}".format(len(attrib.landmark_id.unique())))

So there are a whopping ~200k landmarks in this dataset. My house is probably also listed here. Now I don't want to use all these 200k categories. Majority of them will have very examples of it. So I'll exclude categories that have less than a certain threshold number of images to it's name.

In [ ]:
NUM_THRESHOLD = 250

counts = dict(Counter(train['landmark_id']))
landmarks_dict = {x:[] for x in train.landmark_id.unique() if counts[x] >= NUM_THRESHOLD}
NUM_CLASSES = len(landmarks_dict)
print("Total number of valid classes: {}".format(NUM_CLASSES))

i = 0
landmark_to_idx = {}
idx_to_landmark = []
for k in landmarks_dict:
    landmark_to_idx[k] = i
    idx_to_landmark.append(k)
    i += 1

all_urls = train['url'].tolist()
all_landmarks = train['landmark_id'].tolist()
valid_urls_dict = {x[0].split("/")[-1]:landmark_to_idx[x[1]] for x in zip(all_urls, all_landmarks) if x[1] in landmarks_dict}
valid_urls_list = [x[0] for x in zip(all_urls, all_landmarks) if x[1] in landmarks_dict]

NUM_EXAMPLES = len(valid_urls_list)
print("Total number of valid examples: {}".format(NUM_EXAMPLES))

Let us check a few sample images from the dataset. First we'll check a few images overall. In the next plot, we'll see how similar are the images from the same categories

In [ ]:
w=20
h=20
fig=plt.figure(figsize=(16, 16))
columns = 4
rows = 4
i = 1
for url in valid_urls_list[:16]:
    im = download_image_cv2_urllib(url)
    if im.size != 0:
        fig.add_subplot(rows, columns, i)
        plt.title("Landmark: "+str(idx_to_landmark[valid_urls_dict[url.split("/")[-1]]]))
        plt.imshow(im)
        i += 1

In [ ]:
w=20
h=20
fig=plt.figure(figsize=(16, 16))
columns = 5
rows = 4
landmarks = [idx_to_landmark[valid_urls_dict[x]] for x in random.sample(valid_urls_dict.keys(), rows)]
for i in range(rows):
    landmark = landmarks[i]
    urls = [x[0] for x in zip(all_urls, all_landmarks) if x[1]==landmark]
    for j in range(columns):
        im = download_image_cv2_urllib(urls[j])
        if im.size != 0:
            fig.add_subplot(rows, columns, i*columns+j+1)
            plt.title("Landmark: "+str(landmark))
            plt.imshow(im)